# Testing

#### Modules

In [1]:
#import pyrtp.pwd as pwd
import win32com.client
from tkinter import filedialog
import tkinter as tk
import pandas as pd
import os

#### Import PW/Aux Files

In [2]:
#pw_file = filedialog.askopenfilename(title="Select your PowerWorld case", filetypes= [("PowerWorld files", "*.pwb")])
pw_file = '23SSWG_2030_SUM1_U1_Final_10092023_v220.pwb'

#### COM Connection and open PW case

In [3]:
simauto_obj = win32com.client.Dispatch('pwrworld.SimulatorAuto')
simauto_obj.OpenCase(pw_file)

('OpenCase: Errors have occurred',)

# Functions

#### Open PW Case

In [4]:
def open_pw(pw_file):
    # Create PowerWorld COM instance
    simauto_obj = win32com.client.Dispatch('pwrworld.SimulatorAuto')
    
    # Open the PowerWorld case
    simauto_obj.OpenCase(pw_file)
    
    print(f"PowerWorld case '{os.path.basename(pw_file)}' opened successfully.")
    
    return simauto_obj

#### Close PW Case

In [5]:
def close_pw(simauto_obj):
    simauto_obj.CloseCase()
    print(f"PowerWorld case closed successfully.")

#### Select PW & Aux Cases

In [6]:
def select_pw() -> str:
    root = tk.Tk()
    root.withdraw()  # Hide the root window
    root.attributes('-topmost', True)  # Make the root window topmost
    pw_file = filedialog.askopenfilename(title="Select PowerWorld File", filetypes=[("PowerWorld Cases", "*.pwb")])
    root.attributes('-topmost', False)  # Reset the topmost attribute
    root.destroy()  # Destroy the root window
    return pw_file


def select_aux() -> str:
    root = tk.Tk()
    root.withdraw()  # Hide the root window
    root.attributes('-topmost', True)  # Make the root window topmost
    aux_file = filedialog.askopenfilename(title="Select Auxiliary File", filetypes=[("Auxiliary Files", "*.aux")])
    root.attributes('-topmost', False)  # Reset the topmost attribute
    root.destroy()  # Destroy the root window
    return aux_file


def select_pw_aux_multiple() -> tuple:
    root = tk.Tk()
    root.withdraw()  # Hide the root window
    root.attributes('-topmost', True)  # Make the root window topmost
    pw_files = filedialog.askopenfilenames(title="Select PowerWorld Files", filetypes=[("PowerWorld Cases", "*.pwb")])
    aux_files = filedialog.askopenfilenames(title="Select Aux Files", filetypes=[("Auxiliary Files", "*.aux")])
    root.attributes('-topmost', False)  # Reset the topmost attribute
    root.destroy()  # Destroy the root window
    return pw_files, aux_files


#### Use os.path to manipulate the path of pw file

In [7]:
def get_file_path_info(file: str) -> tuple:
    directory = os.path.dirname(file)
    file_name = os.path.basename(file)
    root, ext = os.path.splitext(file)

    return directory, file_name, root, ext

directory, file_name, root, ext = get_file_path_info(pw_file)
print(pw_file)
print(f"Directory: {directory} \nFile Name: {file_name} \nRoot: {root} \nExtension: {ext}")

23SSWG_2030_SUM1_U1_Final_10092023_v220.pwb
Directory:  
File Name: 23SSWG_2030_SUM1_U1_Final_10092023_v220.pwb 
Root: 23SSWG_2030_SUM1_U1_Final_10092023_v220 
Extension: .pwb


#### Aux File applicator

In [8]:
def aux_file_applicator():

    # Select PW cases and Aux files to be applied
    pw_files, aux_files = select_pw_aux_multiple()

    # Iterate over the PW cases and apply the aux files to each case
    for case in pw_files:
        directory, file_name, root, ext = get_file_path_info(case)
        simauto_obj = open_pw(case)
        # Iterate over the aux files to be applied. Will run Power Flow twice after applying each aux file
        for aux in aux_files:
            simauto_obj.RunScriptCommand("EnterMode(EDIT);")
            print(f"Applying aux file: {os.path.basename(aux)}")
            simauto_obj.ProcessAuxFile(aux)
            simauto_obj.RunScriptCommand("SolvePowerFlow")
            simauto_obj.RunScriptCommand("SolvePowerFlow")
        # Save the log
        log = root + "_log.txt"
        simauto_obj.RunScriptCommand(f'LogSave({log}, NO)')
        print(f"Saved log: {log}")
        # Save the case with the aux files
        case_with_aux = root + "_with_aux.PWB"
        simauto_obj.SaveCase(case_with_aux, "PWB", False)
        print(f"Saved case: {case_with_aux}")
        simauto_obj.CloseCase()

#aux_file_applicator()

#### Test

# SimAuto Documentation

# GitHub Commands

# Miscellaneous

#### Convert Jupyter notebook into python file using command prompt

In [ ]:
#cd (path)
#jupyter nbconvert --to script file.py

#### Run jupyter notebook functions

In [ ]:
#%run my_functions.ipynb